# Part 2 - Programming CKY Algorithm 

In [1]:
from tabulate import tabulate
import nltk

In [2]:
part2_grammar = nltk.CFG.fromstring("""
S -> NP VP
NP -> JJ NP
VP -> VP NP
VP -> VP PP
PP -> P NP
NP -> 'British'
JJ -> 'British'
NP -> 'left'
VP -> 'left'
NP -> 'waffles'
VP -> 'waffles'
P -> 'on'
NP -> 'Falklands'
""")

In [3]:
print('Starting symbol:', part2_grammar.start())
print('Data type of the symbol:', type(part2_grammar.start()))

Starting symbol: S
Data type of the symbol: <class 'nltk.grammar.Nonterminal'>


In [4]:
non_terminals = set()
productions = part2_grammar.productions()
for rule in productions:
    non_terminals.add(rule.lhs())
print("Non-terminals =", non_terminals)
nonterms = list(non_terminals)
print("Rules: ")
for rule in productions:
    print(rule)

Non-terminals = {VP, JJ, S, PP, P, NP}
Rules: 
S -> NP VP
NP -> JJ NP
VP -> VP NP
VP -> VP PP
PP -> P NP
NP -> 'British'
JJ -> 'British'
NP -> 'left'
VP -> 'left'
NP -> 'waffles'
VP -> 'waffles'
P -> 'on'
NP -> 'Falklands'


In [5]:
sentence = "British left waffles on Falklands"
words = sentence.split()
parser = nltk.ChartParser(part2_grammar)
trees = list(parser.parse(words))
print("num of trees =", len(trees))
print(trees[0])
print(trees[1])
#trees[0]
#trees[1]

num of trees = 2
(S
  (NP British)
  (VP (VP (VP left) (NP waffles)) (PP (P on) (NP Falklands))))
(S
  (NP (JJ British) (NP left))
  (VP (VP waffles) (PP (P on) (NP Falklands))))


In [6]:
def cky2(words, grammar):
    n = len(words)
    # Create table with n + 1 rows and columns, with each cell containing empty set
    table = [[set() for i in range(n + 1)] for i in range(n + 1)]
    
    # initialize the diagonal elements of the table
    for i in range(n):
        for production in grammar.productions(rhs=words[i]):
                table[i][i + 1].add(production.lhs())
    
    for width in range(2, n + 1):
        for i in range(0, n - width + 1):
            for j in range(1, width):
                non_term1 = table[i][i + j]
                non_term2 = table[i + j][i + width]
                for nt1 in non_term1:
                    productions = grammar.productions(rhs=nt1)
                    for production in productions:
                        if production.rhs()[1] in non_term2:
                            table[i][i + width].add(production.lhs())

    t2 =[]
    for i in range(n):
        r = []
        for j in range(1, n + 1):
            cell = table[i][j]
            if cell:
                r.append(', '.join([x.symbol() for x in cell]))
            else:
                r.append('-')
        t2.append(r)
        
    print(tabulate(t2, headers=words, tablefmt='grid')) 

In [7]:
# Using NLTK
cky2(words, part2_grammar)

+-----------+--------+-----------+------+-------------+
| British   | left   | waffles   | on   | Falklands   |
+===========+========+===========+======+=============+
| JJ, NP    | NP, S  | S         | -    | S           |
+-----------+--------+-----------+------+-------------+
| -         | VP, NP | VP, S     | -    | VP, S       |
+-----------+--------+-----------+------+-------------+
| -         | -      | VP, NP    | -    | VP          |
+-----------+--------+-----------+------+-------------+
| -         | -      | -         | P    | PP          |
+-----------+--------+-----------+------+-------------+
| -         | -      | -         | -    | NP          |
+-----------+--------+-----------+------+-------------+


In [8]:
non_terminals = ["NP", "VP", "JJ", "PP", "P"]
terminals = ["British", "left", "waffles", "on", "Falklands"]

In [9]:
# Rules of the grammar
R = {
    "S": [["NP", "VP"]],
    "NP": [["JJ", "NP"], ["British"], ["left"], ["waffles"], ["Falklands"]],
    "VP": [["VP", "NP"], ["VP", "PP"], ["left"], ["waffles"]],
    "PP": [["P", "NP"]],
    "JJ": [["British"]],
    "P": [["on"]],
}

In [10]:
# Function to perform the CYK Algorithm
def cykParse(w):
    n = len(w)

    # Initialize the table
    T = [[set() for j in range(n)] for i in range(n)]

    # Filling in the table
    for j in range(n):
        for lhs, rule in R.items():
            for rhs in rule:
                if len(rhs) == 1 and rhs[0] == w[j]:
                    T[j][j].add(lhs)

        for i in range(j, -1, -1):
            for k in range(i, j):
                for lhs, rule in R.items():
                    for rhs in rule:
                        if len(rhs) == 2 and rhs[0] in T[i][k] and rhs[1] in T[k+1][j]:
                            T[i][j].add(lhs)

    # Printing the final parse table
    table = []
    for i in range(n):
        row = []
        for j in range(n):
            row.append(', '.join(T[i][j]) if T[i][j] else '-')
        table.append(row)
    print(tabulate(table, headers=w, tablefmt='grid'))

In [11]:
# Given string
sentence = "British left waffles on Falklands"
T = cykParse(sentence.split())

+-----------+--------+-----------+------+-------------+
| British   | left   | waffles   | on   | Falklands   |
+===========+========+===========+======+=============+
| JJ, NP    | NP, S  | S         | -    | S           |
+-----------+--------+-----------+------+-------------+
| -         | VP, NP | VP, S     | -    | VP, S       |
+-----------+--------+-----------+------+-------------+
| -         | -      | VP, NP    | -    | VP          |
+-----------+--------+-----------+------+-------------+
| -         | -      | -         | P    | PP          |
+-----------+--------+-----------+------+-------------+
| -         | -      | -         | -    | NP          |
+-----------+--------+-----------+------+-------------+


# Part 3 -  Weighted CKY Algorithm

In [12]:
part3_grammar = nltk.PCFG.fromstring("""
S -> NP VP [1.0]
PP -> P NP [1.0]
VP -> V NP [0.7]
VP -> VP PP [0.3]
P -> 'with' [1.0]
V -> 'saw' [1.0]
NP -> NP PP [0.4]
NP -> 'astronomers' [0.1]
NP -> 'ears' [0.18]
NP -> 'saw' [0.04]
NP -> 'stars' [0.18]
NP -> 'telescopes' [0.1]
""")

In [13]:
print(part3_grammar.start())

S


In [14]:
sentence = "astronomers saw stars with ears"
words = sentence.split()
parser = nltk.ChartParser(part3_grammar)
trees = list(parser.parse(words))
print("num of trees =", len(trees))
print(trees[0])
print(trees[1])

num of trees = 2
(S
  (NP astronomers)
  (VP (VP (V saw) (NP stars)) (PP (P with) (NP ears))))
(S
  (NP astronomers)
  (VP (V saw) (NP (NP stars) (PP (P with) (NP ears)))))


In [15]:
# Using NLTK
cky2(words, part3_grammar)

+---------------+-------+---------+--------+--------+
| astronomers   | saw   | stars   | with   | ears   |
+===============+=======+=========+========+========+
| NP            | -     | S       | -      | S      |
+---------------+-------+---------+--------+--------+
| -             | NP, V | VP      | -      | VP     |
+---------------+-------+---------+--------+--------+
| -             | -     | NP      | -      | NP     |
+---------------+-------+---------+--------+--------+
| -             | -     | -       | P      | PP     |
+---------------+-------+---------+--------+--------+
| -             | -     | -       | -      | NP     |
+---------------+-------+---------+--------+--------+


In [16]:
# parser = nltk.parse.ViterbiParser(part3_grammar)
parser = nltk.pchart.InsideChartParser(part3_grammar)
parser.trace(3)
for parse in parser.parse(words):
  print(parse)

  |[-] . . . .| [0:1] 'astronomers'                  [1.0]
  |. [-] . . .| [1:2] 'saw'                          [1.0]
  |. . [-] . .| [2:3] 'stars'                        [1.0]
  |. . . [-] .| [3:4] 'with'                         [1.0]
  |. . . . [-]| [4:5] 'ears'                         [1.0]
  |. . . . [-]| [4:5] 'ears'                         [1.0]
  |. . . [-] .| [3:4] 'with'                         [1.0]
  |. . . [-] .| [3:4] P  -> 'with' *                 [1.0]
  |. . . [-> .| [3:4] PP -> P * NP                   [1.0]
  |. . . > . .| [3:3] PP -> * P NP                   [1.0]
  |. . . > . .| [3:3] P  -> * 'with'                 [1.0]
  |. . [-] . .| [2:3] 'stars'                        [1.0]
  |. [-] . . .| [1:2] 'saw'                          [1.0]
  |. [-] . . .| [1:2] V  -> 'saw' *                  [1.0]
  |. > . . . .| [1:1] V  -> * 'saw'                  [1.0]
  |[-] . . . .| [0:1] 'astronomers'                  [1.0]
  |. [-> . . .| [1:2] VP -> V * NP                   [0.

In [17]:
# Define the PCFG
pcfg = {
    ("S", ("NP", "VP")): 1.0,
    ("PP", ("P", "NP")): 1.0,
    ("VP", ("V", "NP")): 0.7,
    ("VP", ("VP", "PP")): 0.3,
    ("P", "with"): 1.0,
    ("V", "saw"): 1.0,
    ("NP", ("N", "PP")): 0.4,
    ("NP", "astronomers"): 0.4,
    ("NP", "ears"): 0.18,
    ("NP", "stars"): 0.18,
    ("NP", "telescopes"): 0.1
}

In [18]:
def weighted_cyk_parse(sentence, pcfg):
    words = sentence.split()
    n = len(words)

    # Initialize the table for probabilities
    table = [[{} for _ in range(n)] for _ in range(n)]

    # Fill in the table
    for j in range(n):
        for lhs, rhs in pcfg.keys():
            if isinstance(rhs, str) and rhs == words[j]:
                table[j][j][lhs] = pcfg[(lhs, rhs)]

        for i in range(j-1, -1, -1):
            for k in range(i, j):
                for (X, Y), prob in pcfg.items():
                    if isinstance(Y, tuple) and len(Y) == 2:
                        for x in table[i][k]:
                            for y in table[k+1][j]:
                                if (x, y) == Y:
                                    if X not in table[i][j]:
                                        table[i][j][X] = 0.0
                                    table[i][j][X] += table[i][k][x] * table[k+1][j][y] * prob

    # Find the most probable parse tree and its probability
    max_prob = 0.0
    max_parse_tree = None
    for key, value in table[0][n-1].items():
        if value > max_prob:
            max_prob = value
            max_parse_tree = key

    # Print the parse table
    headers = words
    table_data = []
    for i in range(n):
        row = []
        for j in range(n):
            row.append(', '.join([f"{symbol} ({prob:.4f})" for symbol, prob in table[i][j].items()]) if table[i][j] else '-')
        table_data.append(row)

    print("Parse Table:")
    print(tabulate(table_data, headers=headers))

    print("\nMost Probable Parse Tree:", max_parse_tree)
    print("Probability:", max_prob)

In [19]:
sentence = "astronomers saw stars with ears"
weighted_cyk_parse(sentence, pcfg)

Parse Table:
astronomers    saw         stars        with        ears
-------------  ----------  -----------  ----------  -----------
NP (0.4000)    -           S (0.0504)   -           S (0.0027)
-              V (1.0000)  VP (0.1260)  -           VP (0.0068)
-              -           NP (0.1800)  -           -
-              -           -            P (1.0000)  PP (0.1800)
-              -           -            -           NP (0.1800)

Most Probable Parse Tree: S
Probability: 0.0027216


# Part 4 - Marginalize Parse Tree

In [20]:
def marginalize_parse_trees(sentence, pcfg):
    words = sentence.split()
    n = len(words)

    # Initialize the table for probabilities
    table = [[{} for _ in range(n)] for _ in range(n)]

    # Fill in the table
    for j in range(n):
        for lhs, rhs in pcfg.keys():
            if isinstance(rhs, str) and rhs == words[j]:
                table[j][j][lhs] = pcfg[(lhs, rhs)]

        for i in range(j-1, -1, -1):
            for k in range(i, j):
                for (X, Y), prob in pcfg.items():
                    if isinstance(Y, tuple) and len(Y) == 2:
                        for x in table[i][k]:
                            for y in table[k+1][j]:
                                if (x, y) == Y:
                                    if X not in table[i][j]:
                                        table[i][j][X] = 0.0
                                    table[i][j][X] += table[i][k][x] * table[k+1][j][y] * prob

    # Print the parse table
    print("Parse Table:")
    headers = words
    table_data = []
    for i in range(n):
        row = []
        for j in range(n):
            row.append(', '.join([f"{symbol} ({prob:.4f})" for symbol, prob in table[i][j].items()]) if table[i][j] else '-')
        table_data.append(row)
    print(tabulate(table_data, headers=headers))

    # Sum up probabilities of all parse trees
    total_prob = sum(prob for prob in table[0][n-1].values())

    # Print the total probability of the sentence
    print("\nTotal Probability of the Sentence:", total_prob)

In [21]:
sentence = "astronomers saw stars with ears"
marginalize_parse_trees(sentence, pcfg)

Parse Table:
astronomers    saw         stars        with        ears
-------------  ----------  -----------  ----------  -----------
NP (0.4000)    -           S (0.0504)   -           S (0.0027)
-              V (1.0000)  VP (0.1260)  -           VP (0.0068)
-              -           NP (0.1800)  -           -
-              -           -            P (1.0000)  PP (0.1800)
-              -           -            -           NP (0.1800)

Total Probability of the Sentence: 0.0027216
